<a href="https://colab.research.google.com/github/GeorgeTelles/detector_topos_fundos/blob/main/detector_topos_fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://postimg.cc/FYtnXpJw">
  <img src="https://i.postimg.cc/t470PmRR/logo-sem-fundo-01.png" alt="logo" width="300">
</a>

---
# **Detector de topos e fundos**

#### Metodologia utilizando mínimas e máximas do candle
---

#1. Bibliotecas

In [ ]:
!pip install mplfinance

In [ ]:
import pandas as pd
import numpy as np
import pytz
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

#2. Ententendo a metodologia

#### Obtenção de dados OHLC

In [ ]:
df = yf.download('^BVSP', start='2024-01-01', end='2024-07-20')
mpf.plot(df[-30:], type='candle')

#3. Construção das Funções

##3.1. Primeira etapa

Vou retomar os dados diários do IBOV em 2024 e checar o **candle do dia 29/jun, que é um fundo segundo nossa metodologia para 2 candles**

Note que o candle referência atende às 2 regras:
- Mínima (candle referência) é menor que a dos dois candles anteriores,
- Mínima (candle referência) é menor que a dos dois candles posteriores

In [ ]:
mpf.plot(df[-9:], type='candle')

In [ ]:
mpf.plot(df[:10], type='candle')

In [ ]:
df_invertido = df.iloc[::-1]
df_invertido['Low'] <= df_invertido['Low'].rolling(2, closed='left').min()

In [ ]:
def localizador_fundos(df, n_candles=2):
  df_invertido = df.iloc[::-1]

  cond_low_anteriores = df['Low'] <= df['Low'].rolling(n_candles, closed='left').min()
  cond_low_posteriores = df_invertido['Low'] <= df_invertido['Low'].rolling(n_candles, closed='left').min()

  return (cond_low_anteriores & cond_low_posteriores)

def localizador_topos(df, n_candles=2):
  df_invertido = df.iloc[::-1]

  cond_high_anteriores = df['High'] <= df['High'].rolling(n_candles, closed='left').max()
  cond_high_posteriores = df_invertido['High'] <= df_invertido['High'].rolling(n_candles, closed='left').max()

  return (cond_high_anteriores & cond_high_posteriores)

In [ ]:
localizador_fundos(df)

In [ ]:
localizador_topos(df)

In [ ]:
n_candles = 2
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

for i in range(len(df)):

  if localizador_fundos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'fundo':
      dict_topos_fundos['vertice'].append('fundo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.Low.iloc[i])
      ls_vertices.append('fundo')

    else:
      if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

  if localizador_topos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'topo':
      dict_topos_fundos['vertice'].append('topo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.High.iloc[i])
      ls_vertices.append('topo')

    else:
      if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.High.iloc[i]

In [ ]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(
    name='',
    x=df_figure.index,
    open=df_figure.Open,
    high=df_figure.High,
    low=df_figure.Low,
    close=df_figure.Close,
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(
    x=df_topos_fundos.data,
    y=df_topos_fundos.preco,
    mode='lines',
    line_width=2,
    line_dash='solid',
    line_color='black',
    name='Topos / Fundos',
    showlegend=True
))

fig_topos_fundos.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])

fig_topos_fundos.update_layout(
    xaxis_rangeslider_visible=False,
    title_text='<b>Indicador Topos e Fundos',
    template='none',
    margin=dict(l=75, r=75, t=75, b=75),
    paper_bgcolor='#f7f8fa',
    width=1200,
    height=600
)

In [ ]:
n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

for i in range(len(df)):

  if localizador_fundos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'fundo':
      dict_topos_fundos['vertice'].append('fundo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.Low.iloc[i])
      ls_vertices.append('fundo')

    else:
      if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

  if localizador_topos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'topo':
      dict_topos_fundos['vertice'].append('topo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.High.iloc[i])
      ls_vertices.append('topo')

    else:
      if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.High.iloc[i]


df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(
    name='',
    x=df_figure.index,
    open=df_figure.Open,
    high=df_figure.High,
    low=df_figure.Low,
    close=df_figure.Close,
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(
    x=df_topos_fundos.data,
    y=df_topos_fundos.preco,
    mode='lines',
    line_width=2,
    line_dash='solid',
    line_color='black',
    name='Topos / Fundos',
    showlegend=True
))

fig_topos_fundos.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])

fig_topos_fundos.update_layout(
    xaxis_rangeslider_visible=False,
    title_text='<b>Indicador Topos e Fundos',
    template='none',
    margin=dict(l=75, r=75, t=75, b=75),
    paper_bgcolor='#f7f8fa',
    width=1200,
    height=600
)

#4. Teoria de Dow

In [ ]:
df_1w = yf.download('^BVSP', start='2020-01-01', end='2024-07-20', interval='1wk')
df_1d = yf.download('^BVSP', start='2020-01-01', end='2024-07-20', interval='1d')

In [ ]:
df = df_1w.copy()

n_candles = 10
ls_vertices = ['neutro']


dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

for i in range(len(df)):

  if localizador_fundos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'fundo':
      dict_topos_fundos['vertice'].append('fundo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.Low.iloc[i])
      ls_vertices.append('fundo')

    else:
      if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

  if localizador_topos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'topo':
      dict_topos_fundos['vertice'].append('topo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.High.iloc[i])
      ls_vertices.append('topo')

    else:
      if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.High.iloc[i]


df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(
    name='',
    x=df_figure.index,
    open=df_figure.Open,
    high=df_figure.High,
    low=df_figure.Low,
    close=df_figure.Close,
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(
    x=df_topos_fundos.data,
    y=df_topos_fundos.preco,
    mode='lines',
    line_width=2,
    line_dash='solid',
    line_color='black',
    name='Topos / Fundos',
    showlegend=True
))

fig_semanal = fig_topos_fundos.update_layout(
    xaxis_rangeslider_visible=False,
    title_text='<b>Indicador Topos e Fundos',
    template='none',
    margin=dict(l=75, r=75, t=75, b=75),
    paper_bgcolor='#f7f8fa',
    width=1200,
    height=600)

In [ ]:
df = df_1d.copy()

n_candles = 10
ls_vertices = ['neutro']


dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

for i in range(len(df)):

  if localizador_fundos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'fundo':
      dict_topos_fundos['vertice'].append('fundo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.Low.iloc[i])
      ls_vertices.append('fundo')

    else:
      if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

  if localizador_topos(df, n_candles).iloc[i]:

    if ls_vertices[-1] != 'topo':
      dict_topos_fundos['vertice'].append('topo')
      dict_topos_fundos['data'].append(df.index[i])
      dict_topos_fundos['preco'].append(df.High.iloc[i])
      ls_vertices.append('topo')

    else:
      if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
        dict_topos_fundos['data'][-1] = df.index[i]
        dict_topos_fundos['preco'][-1] = df.High.iloc[i]


df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(
    name='',
    x=df_figure.index,
    open=df_figure.Open,
    high=df_figure.High,
    low=df_figure.Low,
    close=df_figure.Close,
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(
    x=df_topos_fundos.data,
    y=df_topos_fundos.preco,
    mode='lines',
    line_width=2,
    line_dash='solid',
    line_color='black',
    name='Topos / Fundos',
    showlegend=True
))

fig_diario = fig_topos_fundos.update_layout(
    xaxis_rangeslider_visible=False,
    title_text='<b>Indicador Topos e Fundos',
    template='none',
    margin=dict(l=75, r=75, t=75, b=75),
    paper_bgcolor='#f7f8fa',
    width=1200,
    height=600)

In [ ]:
fig_semanal

In [ ]:
fig_diario